**Sommario**

- [Query di selezione con Flask-SQLAlchemy](#query-di-selezione-con-flask-sqlalchemy)
  - [Nuova API con `db.session.execute(select(...))`](#nuova-api-con-dbsessionexecuteselect)
    - [`db.session.get()`: una riga della tabella in base alla chiave primaria](#dbsessionget-una-riga-della-tabella-in-base-alla-chiave-primaria)
    - [`select()`: ottenere un oggetto `Select`](#select-ottenere-un-oggetto-select)
    - [`db.session.execute(select())`: ottenere un oggetto `Result`](#dbsessionexecuteselect-ottenere-un-oggetto-result)
    - [`Result.scalars().all()`: tutti i risultati](#resultscalarsall-tutti-i-risultati)
    - [`Result.scalar()`: solo il primo risultato](#resultscalar-solo-il-primo-risultato)
    - [`Result.scalar_one()`: solo un risultato](#resultscalar_one-solo-un-risultato)
    - [`select().limit()`: numero di risultati limitato a una quantità specificata](#selectlimit-numero-di-risultati-limitato-a-una-quantit%C3%A0-specificata)
    - [`select().filter_by()`: risultati filtrati basandosi su parametri chiave-valore](#selectfilter_by-risultati-filtrati-basandosi-su-parametri-chiave-valore)
    - [`select().where()`: risultati filtrati basandosi su una condizione specificata](#selectwhere-risultati-filtrati-basandosi-su-una-condizione-specificata)
    - [`select().order_by()`: risultati ordinati in base a uno o più criteri](#selectorder_by-risultati-ordinati-in-base-a-uno-o-pi%C3%B9-criteri)
    - [`select(func.count())`: numero di risultati](#selectfunccount-numero-di-risultati)
    - [`select().distinct()`: risultati distinti senza duplicati](#selectdistinct-risultati-distinti-senza-duplicati)
    - [`select().join()`: INNER JOIN tra tabelle](#selectjoin-inner-join-tra-tabelle)
    - [`select().outerjoin()`: OUTER JOIN tra tabelle](#selectouterjoin-outer-join-tra-tabelle)
  - [Oggetti *Model*, `Row` e "valori scalari"](#oggetti-model-row-e-valori-scalari)
    - [Oggetti *Model* per rappresentare un record del modello ORM](#oggetti-model-per-rappresentare-un-record-del-modello-orm)
    - [Oggetti `Row` per rappresentare le righe di una query](#oggetti-row-per-rappresentare-le-righe-di-una-query)
    - [`Result.scalars()` e `Result.scalar()`: risultati come valori scalari](#resultscalars-e-resultscalar-risultati-come-valori-scalari)
    - [Scalare Vs `Row`](#scalare-vs-row)
    - [`Row` con Model](#row-con-model)

# Query di selezione con Flask-SQLAlchemy

## Nuova API con `db.session.execute(select(...))`

La nuova sintassi per le query è stata introdotta con SQLAlchemy 2.0.

Leggi la [guida per la migrazione dalle vecchie API alle nuove](https://docs.sqlalchemy.org/en/20/changelog/migration_20.html#migration-20-query-usage).

Per approfondire: [Nuove Select API nella documentazione ufficiale](https://docs.sqlalchemy.org/en/20/core/selectable.html).

### `db.session.get()`: una riga della tabella in base alla chiave primaria

```python
from myapp import db
from myapp.models import User

user = db.session.get(User, 1)
```

OUTPUT: Restituisce l'oggetto `User` con la chiave primaria uguale a 1. Se non esiste, restituisce `None`.

### `select()`: ottenere un oggetto `Select`

La funzione select() è un costruttore fornito da SQLAlchemy per creare un'istruzione SQL `SELECT`. Questa funzione viene utilizzata per costruire query che selezionano dati da una o più tabelle.

OUTPUT: Restituisce un oggetto `Select` che può essere ulteriormente modificato concatenandovi altri metodi e poi eseguito con `db.session.execute()` per ottenere i risultati (`Result`) desiderati.

### `db.session.execute(select())`: ottenere un oggetto `Result`

Il metodo `.execute()` di `db.session` consente di eseguire istruzioni SQL arbitrarie. Può essere utilizzato per eseguire query di selezione, aggiornamento, cancellazione e inserimento direttamente nel database. 

OUTPUT: Restituisce un oggetto `Result`, che è un contenitore per i risultati della query eseguita. Questo oggetto fornisce vari metodi per accedere ai risultati in modi diversi, come `.all()`, `.first()`, `.scalars()`, `.scalar()` e `.scalar_one()`.

Vedi le [Session API](https://docs.sqlalchemy.org/en/20/orm/session_api.html) per approfondire.

Per eseguire query di selezione dobbiamo passare un oggetto `Select` (creato con il costruttore `select()`) a `db.session.execute()` ed ottenere così un oggetto `Result`:

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

# Costruisce una query di selezione
stmt = select(User).where(User.age > 18)  # Select object

# Esegui la query
result = db.session.execute(stmt)  # Result object
```

> *NOTA*: `stmt` è una abbreviazione convenzionale per "*statement*" (istruzione) nel contesto della programmazione SQL.

### `Result.scalars().all()`: tutti i risultati

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User)  # SELECT * FROM user
result = db.session.execute(stmt).scalars().all()
```

OUTPUT: Restituisce una lista di oggetti `User` contenenti tutte le righe della tabella `User`.

### `Result.scalar()`: solo il primo risultato

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User)  # SELECT * FROM user
result = db.session.execute(stmt).scalar()
# equivalente a
result = db.session.execute(stmt).scalars().first()
```

OUTPUT: Restituisce il primo oggetto `User` della tabella `User`. Se la tabella è vuota, restituisce `None`.

### `Result.scalar_one()`: solo un risultato

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User)  # SELECT * FROM user
result = db.session.execute(stmt).scalar_one()
# equivalente a
result = db.session.execute(stmt).scalars().one()
```

OUTPUT: Se la tabella `User` contiene una sola riga la restituisce, altrimenti solleva un errore sia se ci sono più ricghe sia se non ce ne sono.

### `select().limit()`: numero di risultati limitato a una quantità specificata

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User).limit(10)
result = db.session.execute(stmt).scalars().all()
```
OUTPUT: Restituisce una lista di al massimo 10 oggetti `User`.

### `select().filter_by()`: risultati filtrati basandosi su parametri chiave-valore

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User).filter_by(name='Alice')
result = db.session.execute(stmt).scalars().all()
```

OUTPUT: Restituisce una lista di oggetti `User` che hanno il nome 'Alice'.

### `select().where()`: risultati filtrati basandosi su una condizione specificata

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User).where(User.age > 18)
result = db.session.execute(stmt).scalars().all()
```

OUTPUT: Restituisce una lista di oggetti `User` che soddisfano la condizione specificata (in questo caso, utenti con età superiore a 18).

### `select().order_by()`: risultati ordinati in base a uno o più criteri

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User).order_by(User.name)
result = db.session.execute(stmt).scalars().all()
```

OUTPUT: Restituisce una lista di oggetti `User` ordinati per nome in ordine crescente.

### `select(func.count())`: numero di risultati

```python
from sqlalchemy import select, func
from myapp import db
from myapp.models import User

stmt = select(func.count(User.id)).where(User.age > 18)
result = db.session.execute(stmt).scalar()
```

OUTPUT: Restituisce un numero intero rappresentante il conteggio degli utenti con età superiore a 18.

### `select().distinct()`: risultati distinti senza duplicati

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User

stmt = select(User.name).distinct()
result = db.session.execute(stmt).scalars().all()
```

OUTPUT: Restituisce una lista di nomi distinti degli utenti.

### `select().join()`: INNER JOIN tra tabelle

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User, Post

stmt = select(User, Post).join(Post, User.id == Post.user_id)
result = db.session.execute(stmt).all()
```

OUTPUT: Restituisce una lista di tuple contenenti oggetti `User` e `Post` uniti tramite la condizione specificata.

### `select().outerjoin()`: OUTER JOIN tra tabelle

```python
from sqlalchemy import select
from myapp import db
from myapp.models import User, Post

stmt = select(User, Post).outerjoin(Post, User.id == Post.user_id)
result = db.session.execute(stmt).all()
```

OUTPUT: Restituisce una lista di tuple contenenti oggetti `User` e `Post` uniti tramite una `LEFT OUTER JOIN`, secondo la condizione specificata. Se un utente non ha post corrispondenti, le colonne della tabella `Post` saranno `NULL`.

> **NOTA BENE**: La `LEFT OUTER JOIN` mostra tutte le righe della tabella di sinistra (`User`), e le righe corrispondenti della tabella di destra (`Post`). Se non ci sono corrispondenze, le colonne della tabella di destra saranno `NULL`.

## Oggetti *Model*, `Row` e "valori scalari"

### Oggetti *Model* per rappresentare un record del modello ORM

Gli oggetti *Model* sono istanze delle classi definite nel modello ORM (Object-Relational Mapping) che ereditano la metaclasse `db.Model`.

Queste classi mappano direttamente alle tabelle del database e forniscono un'interfaccia orientata agli oggetti per interagire con i dati. Inoltre espongono i metodi implementati nella classe.

I valori delle colonne di un oggetto Model sono accessibili come attributi in dot.notation.

Ideali per applicazioni che beneficiano di un'interfaccia orientata agli oggetti, logiche di business e relazioni tra tabelle.

### Oggetti `Row` per rappresentare le righe di una query

Gli oggetti `Row` (di tipo `sqlalchemy.engine.row.Row`) rappresentano singole righe dei risultati di una query eseguita direttamente sul database. 

L'oggetto `Row` quando viene stampato è rappresentato come una tupla di valori.

I valori di un oggetto `Row` sono accessibili tramite indici o attributi in dot.notation.

Utili per operazioni semplici, accesso rapido ai dati e quando la performance è una priorità.


### `Result.scalars()` e `Result.scalar()`: risultati come valori scalari

Dopo aver eseguito la query con `db.session.execute()` otteniamo un oggetto `Result`. Solitamente volgliamo estrarre i risultati dall'oggetto `Result` ottenuto, in modo da ottenere una lista con gli oggetti o i valori richiesti.

Se si desidera ottenere una lista di valori scalari (singoli) anziché una lista di tuple (oggetti `Row`) con un singolo elemento, la nuova API per le query richiede di passare i risultati attraverso il metodo `.scalars()` prima di applicare `.all()` e altri metodi simili:

- `Result.all()` restituisce una lista di tuple (`Row`);
- `Result.scalars().all()` restituisce una lista di valori scalari.

Se invece si vuole ottenere la prima riga oppure `None` in caso non ci siano risultati, possiamo usare:

- `Result.first()` restituisce la prima riga come singola tupla (`Row`) o `None`;
- `Result.scalars().first()` restituisce la prima riga come singolo valore scalare o `None`;
- `Result.scalar()` restituisce la prima riga come singolo valore scalare o `None`.

Se infine si vuole ottenere e verificare la presenza di una e un sola riga, possiamo usare:

- `Result.one()` restituisce una singola tupla (`Row`);
- `Result.scalars().one()` restituisce un singolo valore scalare;
- `Result.scalar_one()` restituisce un singolo valore scalare.

> **ATTENZIONE**: `.scalars()`, `.scalar()` e `.scalar_one()` hanno senso solo se in `select()` si è selezionato l'intero modello o una singola colonna. Se in `select()` vengono passate più colonne, restituirà solamente i valori appartenenti alla prima colonna!

> **NOTA**: In questo contesto, un _**valore scalare**_ si riferisce a un singolo valore/oggetto elementare estratto dai risultati di una query. Questo è in contrasto con l'oggetto `Row` che è simile a una tupla di valori (accessibili tramite indici o attributi in dot.notation), che è considerata un oggetto non scalare.

### Scalare Vs `Row`

Se abbiamo questa tabella `User`:

| id | name  | surname  | age |
|----|-------|----------|-----|
| 1  | Alice | Simpson  | 30  |
| 2  | Bob   | Smith    | 25  |
| 3  | Carol | Tyler    | 22  |

```python
stmt = select(User.name, User.surname, User.age)  # Select
result = db.session.execute(stmt)  # Result

result.all()         # list of Row : [('Alice', 'Simpson', 30),
                     #                ('Bob', 'Smith', 25),
                     #                ('Carol', 'Tyler', 22)]
result.all()[0]      #  Row : ('Alice', 'Simpson', 30)
result.all()[0][2]   #  int : 30
result.all()[0].age  #  int : 30

result.scalars().all()  # list of str : ['Alice', 'Bob', 'Carol']

result.first()      # one Row or None : ('Alice', 'Simpson', 30)
result.first()[2]   # int : 30
result.first().age  # int : 30

result.scalar()  # string or None : 'Alice'

result.one()  # Row or error : ('Alice', 'Simpson', 30)

result.scalar_one()  # string or error : 'Alice'
```

### `Row` con Model

Se selezioniamo l'intero oggetto Model con `select(User)` anziché specificare i nomi delle colonne, i risultati conterranno direttamente gli oggetti Model all'interno delle `Row`.

Veriamo alcuni esempi per provare a capire un po' meglio. Usiamo sempre la stessa tabella `User`:

| id | name  | surname  | age |
|----|-------|----------|-----|
| 1  | Alice | Simpson  | 30  |
| 2  | Bob   | Smith    | 25  |
| 3  | Carol | Tyler    | 22  |

```python
stmt = select(User)  # Select
result = db.session.execute(stmt)  # Result

result.all()            # list of Row : [(<User 1>,), (<User 2>,), (<User 3>,)]
result.all()[0]         # Row : (<User 1>,)
result.all()[0][0]      # Model : <User 1>
result.all()[0][0].age  # int : 30

result.scalars().all()  # list of Model : [<User 1>, <User 2>, <User 3>]

result.first()     # Row or None : (<User 1>,)
result.first()[0]  # Model : <User 1>

result.scalar()  # Model or None : <User 1>

result.one()  # Row or error : (<User 1>,)

result.scalar_one()  # Model or error : <User 1>